In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import numpy as np

from bayes_opt import BayesianOptimization

from isic_helper import get_folds
from isic_helper import compute_pauc, compute_auc

In [2]:
id_column = "isic_id"
target_column = "target"
ensemble_column = "oof_preds_ensemble"

# model_names = ["efficientnet_b0", "mobilevitv2_200", "boosting"]
# versions = ["v3", "v1", "v1"]
# model_names = ["lgb", "cb", "efficientnet_b0", "mobilevitv2_200"]
# versions = ["v5", "v2", "v3", "v1"]
model_names = ["lgb", "cb", "xgb"]
versions = ["v5", "v2", "v1"]
# model_names = ["lgb", "cb", "xgb", "efficientnet_b0", "mobilevitv2_200"]
# versions = ["v5", "v2", "v1", "v3", "v1"]
paths = [f"/kaggle/input/isic-scd-{model_name.replace('_', '-')}-{version}-train" for model_name, version in zip(model_names, versions)]

In [3]:
val_auc_scores = {}
val_pauc_scores = {}
for idx, path in enumerate(paths):
    model_name = model_names[idx]
    version = versions[idx]
    oof_preds_model_df = pd.read_csv(f"{path}/oof_preds_{model_name}_{version}.csv")
    if idx == 0:
        oof_preds_df = oof_preds_model_df.copy()
    else:
        oof_preds_df = oof_preds_df.merge(oof_preds_model_df[[id_column, f"oof_{model_name}_{version}"]],
                                          on="isic_id", how="inner")
        assert oof_preds_df.shape[0] == oof_preds_model_df.shape[0]
    val_auc_scores[f"{model_name}_{version}"] = {}
    val_pauc_scores[f"{model_name}_{version}"] = {}
val_auc_scores["ensemble"] = {}
val_pauc_scores["ensemble"] = {}

In [4]:
oof_columns = [col for col in oof_preds_df.columns if col.startswith("oof")]
all_folds = np.unique(oof_preds_df["fold"])

In [5]:
def blend_optimizer(oof_preds_df, oof_columns, init_points = 20, n_iter = 100):
    pbounds = {f"w{i}": (0.0, 1.0) for i in range(len(oof_columns))}

    def dim_opt(oof_preds_df, *args):
        weights = args
        score = 0
        for fold in all_folds:
            fold_ensemble_preds = 0
            for weight, oof_column in zip(weights, oof_columns):
                fold_ensemble_preds += weight * oof_preds_df.loc[oof_preds_df["fold"] == fold, oof_column].rank(pct=True).values
            score += compute_pauc(np.array(oof_preds_df.loc[oof_preds_df["fold"] == fold, "target"]), fold_ensemble_preds)
        return score / len(all_folds)
    
#     def dim_opt(oof_preds_df, *args):
#         weights = args
#         score = 0
#         for fold in all_folds:
#             oof_fold_preds_df = oof_preds_df[oof_preds_df["fold"] == fold]
#             fold_ensemble_preds = np.zeros((oof_fold_preds_df.shape[0], len(weights)))
#             for idx, (weight, oof_column) in enumerate(zip(weights, oof_columns)):
#                 fold_ensemble_preds[:, idx] = weight * np.log1p(oof_fold_preds_df[oof_column].rank(pct=True).values)
#             fold_ensemble_preds = np.exp(fold_ensemble_preds.sum(axis=1) / sum(weights)) - 1
#             score += compute_pauc(oof_fold_preds_df["target"], fold_ensemble_preds)
#         return score / len(all_folds)
    
#     def dim_opt(oof_preds_df, *args):
#         weights = args
#         score = 0
#         for fold in all_folds:
#             oof_fold_preds_df = oof_preds_df[oof_preds_df["fold"] == fold]
#             fold_ensemble_preds = np.zeros((oof_fold_preds_df.shape[0], len(weights)))
#             for idx, (weight, oof_column) in enumerate(zip(weights, oof_columns)):
#                 fold_ensemble_preds[:, idx] = weight * oof_fold_preds_df[oof_column].rank(pct=True).values
#             fold_ensemble_preds = fold_ensemble_preds.sum(axis=1) ** (1 / sum(weights))
#             score += compute_pauc(oof_fold_preds_df["target"], fold_ensemble_preds)
#         return score / len(all_folds)

    def q(**ws):
        ws = tuple(ws.values())
        return dim_opt(oof_preds_df, *ws)

    optimizer = BayesianOptimization(
        f=q,
        pbounds=pbounds,
        random_state=2022,
    )

    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )
    
    weights = []
    for i in range(len(oof_columns)):
        weights.append(optimizer.max["params"][f"w{i}"])
    
    print(f"Best pAUC: {optimizer.max['target']}")
    print(f"Best weights: {weights}")
    return weights


weights = blend_optimizer(
    oof_preds_df, oof_columns, 
    init_points=40, 
    n_iter=40
)

|   iter    |  target   |    w0     |    w1     |    w2     |
-------------------------------------------------------------
| 1         | 0.174     | 0.009359  | 0.4991    | 0.1134    |
| 2         | 0.1752    | 0.04997   | 0.6854    | 0.487     |
| 3         | 0.176     | 0.8977    | 0.6475    | 0.897     |
| 4         | 0.1759    | 0.7211    | 0.8314    | 0.8276    |
| 5         | 0.1755    | 0.8336    | 0.957     | 0.368     |
| 6         | 0.176     | 0.4948    | 0.3395    | 0.6194    |
| 7         | 0.1758    | 0.9775    | 0.09643   | 0.7442    |
| 8         | 0.1759    | 0.2925    | 0.2987    | 0.7525    |
| 9         | 0.1755    | 0.01866   | 0.5237    | 0.8644    |
| 10        | 0.176     | 0.3888    | 0.2122    | 0.4752    |
| 11        | 0.176     | 0.5647    | 0.3494    | 0.9759    |
| 12        | 0.1748    | 0.03782   | 0.7943    | 0.3579    |
| 13        | 0.1755    | 0.748     | 0.9145    | 0.3727    |
| 14        | 0.1746    | 0.9649    | 0.08139   | 0.04245   |
| 15    

In [6]:
weights

[0.6854584318985072, 0.3949315008084686, 0.9138840820471673]

In [7]:
all_folds = np.unique(oof_preds_df["fold"])
for fold in all_folds:
    fold_index = oof_preds_df[oof_preds_df["fold"] == fold].index
    fold_target = oof_preds_df.loc[fold_index, target_column]
    fold_ensemble_preds = 0
    for model_name, version, weight in zip(model_names, versions, weights):
        fold_model_preds = oof_preds_df.loc[fold_index, f"oof_{model_name}_{version}"]
        fold_ensemble_preds += fold_model_preds.rank(pct=True).values * weight 
        
        val_auc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_auc(fold_target, fold_model_preds)
        val_pauc_scores[f"{model_name}_{version}"][f"fold_{fold}"] = compute_pauc(fold_target, fold_model_preds, min_tpr=0.8)
    
    oof_preds_df.loc[fold_index, ensemble_column] = fold_ensemble_preds
    val_auc_scores["ensemble"][f"fold_{fold}"] = compute_auc(fold_target, fold_ensemble_preds)
    val_pauc_scores["ensemble"][f"fold_{fold}"] = compute_pauc(fold_target, fold_ensemble_preds, min_tpr=0.8)

for model_name, version, weight in zip(model_names, versions, weights):
    print(f"Model: {model_name}_{version} | Weightage: {weight}")
    
    print("Val AUC scores:")
    pprint(val_auc_scores[f"{model_name}_{version}"])
    print("Val PAUC scores:")
    pprint(val_pauc_scores[f"{model_name}_{version}"])
    
    cv_model_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"])
    cv_model_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[f"oof_{model_name}_{version}"], min_tpr=0.8)

    cv_model_auc_avg = np.mean(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_avg = np.mean(list(val_pauc_scores[f"{model_name}_{version}"].values()))

    cv_model_auc_std = np.std(list(val_auc_scores[f"{model_name}_{version}"].values()))
    cv_model_pauc_std = np.std(list(val_pauc_scores[f"{model_name}_{version}"].values()))
    
    print(f"CV AUC OOF: {cv_model_auc_oof}")
    print(f"CV PAUC OOF: {cv_model_pauc_oof}")
    print(f"CV AUC AVG: {cv_model_auc_avg}")
    print(f"CV PAUC AVG: {cv_model_pauc_avg}")
    print(f"CV AUC STD: {cv_model_auc_std}")
    print(f"CV PAUC STD: {cv_model_pauc_std}")
    print("\n")

print("Val AUC scores:")
pprint(val_auc_scores["ensemble"])
print("Val PAUC scores:")
pprint(val_pauc_scores["ensemble"])

cv_ensemble_auc_oof = compute_auc(oof_preds_df[target_column], oof_preds_df[ensemble_column])
cv_ensemble_pauc_oof = compute_pauc(oof_preds_df[target_column], oof_preds_df[ensemble_column], min_tpr=0.8)

cv_ensemble_auc_avg = np.mean(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_avg = np.mean(list(val_pauc_scores["ensemble"].values()))

cv_ensemble_auc_std = np.std(list(val_auc_scores["ensemble"].values()))
cv_ensemble_pauc_std = np.std(list(val_pauc_scores["ensemble"].values()))

print(f"CV AUC OOF: {cv_ensemble_auc_oof}")
print(f"CV PAUC OOF: {cv_ensemble_pauc_oof}")
print(f"CV AUC AVG: {cv_ensemble_auc_avg}")
print(f"CV PAUC AVG: {cv_ensemble_pauc_avg}")
print(f"CV AUC STD: {cv_ensemble_auc_std}")
print(f"CV PAUC STD: {cv_ensemble_pauc_std}")

Model: lgb_v5 | Weightage: 0.6854584318985072
Val AUC scores:
{'fold_1': 0.9798763448762824,
 'fold_2': 0.9566898521898896,
 'fold_3': 0.9700671400581549,
 'fold_4': 0.9681429471740026,
 'fold_5': 0.9689407385862031}
Val PAUC scores:
{'fold_1': 0.18493155147135198,
 'fold_2': 0.16279309080367302,
 'fold_3': 0.17765128352323065,
 'fold_4': 0.1712910733673609,
 'fold_5': 0.1731502786873233}
CV AUC OOF: 0.9669123174545425
CV PAUC OOF: 0.1721090376888892
CV AUC AVG: 0.9687434045769064
CV PAUC AVG: 0.17396345557058795
CV AUC STD: 0.007367255259379475
CV PAUC STD: 0.007300399098764187


Model: cb_v2 | Weightage: 0.3949315008084686
Val AUC scores:
{'fold_1': 0.9797413487843625,
 'fold_2': 0.9566524148973546,
 'fold_3': 0.9646535048857495,
 'fold_4': 0.9677532727658459,
 'fold_5': 0.9680182337888901}
Val PAUC scores:
{'fold_1': 0.18495560119529067,
 'fold_2': 0.16245001161515996,
 'fold_3': 0.17069065031011715,
 'fold_4': 0.17056070684254526,
 'fold_5': 0.1717592899240909}
CV AUC OOF: 0.958776

In [8]:
oof_preds_df

,isic_id,patient_id,fold,target,oof_lgb_v5,oof_cb_v2,oof_xgb_v1,oof_preds_ensemble
0,ISIC_0015670,IP_1235828,4,0,0.002181,0.002783,0.000596,0.849382
1,ISIC_0015845,IP_8170065,1,0,0.954632,0.987227,0.976310,1.994190
2,ISIC_0015864,IP_6724798,5,0,0.001653,0.000849,0.000495,0.364006
3,ISIC_0015902,IP_4111386,2,0,0.001969,0.000953,0.001736,1.252413
4,ISIC_0024200,IP_8313778,1,0,0.003908,0.002442,0.002855,1.614820
...,...,...,...,...,...,...,...,...
401054,ISIC_9999937,IP_1140263,3,0,0.069581,0.046013,0.129081,1.939529
401055,ISIC_9999951,IP_5678181,3,0,0.001908,0.002491,0.025095,0.780328
401056,ISIC_9999960,IP_0076153,2,0,0.007621,0.003001,0.005917,1.704661
401057,ISIC_9999964,IP_5231513,5,0,0.001988,0.000572,0.001599,0.753297


In [9]:
oof_preds_df.pivot_table(index="target", values=oof_columns+["oof_preds_ensemble"], aggfunc="mean")

,oof_cb_v2,oof_lgb_v5,oof_preds_ensemble,oof_xgb_v1
target,,,,
0,0.007976,0.010858,0.996234,0.012617
1,0.357776,0.396488,1.930373,0.329323


In [10]:
oof_preds_df[oof_columns+["oof_preds_ensemble"]].corr()

,oof_lgb_v5,oof_cb_v2,oof_xgb_v1,oof_preds_ensemble
oof_lgb_v5,1.000000,0.891582,0.860048,0.335124
oof_cb_v2,0.891582,1.000000,0.846495,0.293083
oof_xgb_v1,0.860048,0.846495,1.000000,0.287103
oof_preds_ensemble,0.335124,0.293083,0.287103,1.000000
